In [12]:
# Qwen3-4B-Thinking-2507 Setup
# Run a thinking model locally on RTX 5060 with streaming output.

## Step 1: Install Required Libraries

In [13]:
!pip install "transformers>=4.51.0" accelerate bitsandbytes torch -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Step 2: Import Libraries and Check GPU

In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

PyTorch version: 2.8.0+cu128
CUDA available: True
GPU: NVIDIA GeForce RTX 5060 Laptop GPU
VRAM: 8.08 GB


## Step 3: Configure 4-bit Quantization

In [15]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

## Step 4: Load Model and Tokenizer

In [22]:
model_name = "Qwen/Qwen3-4B-Thinking-2507"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

print(f"✓ Model loaded on {model.device}")
print(f"VRAM: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.36s/it]



✓ Model loaded on cuda:0
VRAM: 4.58 GB


## Step 5: Generate Response

In [23]:
def generate_response(prompt, max_new_tokens=2048):
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(**model_inputs, max_new_tokens=max_new_tokens)
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    
    # Parse thinking content (token 151668 is </think>)
    try:
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0
    
    thinking = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
    
    return thinking, content

# Test prompt
prompt = "Solve this step by step: If a train travels 120 km in 2 hours, then stops for 30 minutes, then travels another 90 km in 1.5 hours, what is the average speed for the entire journey?"

thinking, content = generate_response(prompt)
print("THINKING:", thinking)
print("\nCONTENT:", content)

THINKING: Okay, let's see. I need to find the average speed for the entire journey of the train. Hmm, average speed is total distance divided by total time, right? So first, I should figure out the total distance the train traveled and the total time it took, including the stop. 

Let me break it down step by step. 

First part: The train travels 120 km in 2 hours. So that's distance 1: 120 km, time 1: 2 hours.

Then it stops for 30 minutes. Wait, 30 minutes is 0.5 hours, right? So that's a stop time of 0.5 hours.

Next part: It travels another 90 km in 1.5 hours. So distance 2: 90 km, time 2: 1.5 hours.

So total distance would be 120 km + 90 km. Let me calculate that: 120 + 90 = 210 km. Got that.

Total time is the sum of all the time segments. So time for first part: 2 hours, stop time: 0.5 hours, time for second part: 1.5 hours. So total time = 2 + 0.5 + 1.5. Let me add that up. 2 + 0.5 is 2.5, plus 1.5 is 4 hours. So total time is 4 hours.

Wait, average speed is total distance ov

## Step 6: Custom Prompt

In [24]:
# Your custom prompt
prompt = "Explain the concept of gradient descent in machine learning."

thinking, content = generate_response(prompt)
print("THINKING:", thinking)
print("\nCONTENT:", content)

THINKING: Okay, the user wants me to explain gradient descent in machine learning. Hmm, this is a pretty fundamental concept in ML, so I should make sure I get it right without overwhelming them. 

First, I wonder about their background. Are they a complete beginner? Maybe a student? Or someone who's heard the term but needs clarification? Since they didn't specify, I'll assume they want a clear but not too technical explanation. 

I should start with the big picture: why do we even need gradient descent? Because we're optimizing things like loss functions with tons of parameters. Like, imagine having a mountainous landscape and you want to find the lowest point - but you can't see the whole map, you can only take tiny steps. That's the intuition.

Wait, I should emphasize it's not about the gradient itself but the direction of steepest descent. People often confuse "gradient" with "slope" so I should clarify that. Also must mention it's an iterative algorithm - that's crucial. 

Oh! A

## Step 7: Monitor VRAM

In [25]:
allocated = torch.cuda.memory_allocated(0) / 1e9
total = torch.cuda.get_device_properties(0).total_memory / 1e9

print(f"VRAM: {allocated:.2f} / {total:.2f} GB")

VRAM: 4.58 / 8.08 GB
